In [4]:
import numpy as np
import torch
from torch.nn.functional import conv2d as libConv2d
from torch.nn.functional import conv_transpose2d as libConv2dT

In [107]:
# Создание тензора с исходными данными
input_data = torch.randn(1, 1, 2, 2)  # (batch_size, channels, height, width)

# Создание тензора с весами (ядром) для транспонированной свертки
weights = torch.randn(1, 1, 2, 2)  # (out_channels, in_channels, kernel_height, kernel_width)

In [108]:
# Применение транспонированной свертки
output = libConv2dT(input_data, weights, stride=1, padding=0, dilation=1)

output

tensor([[[[-0.2771,  0.4301,  0.5502],
          [ 1.6777, -0.5814,  0.6351],
          [-1.5935,  2.7090, -0.7724]]]])

In [109]:
stride = 1
padding = 0
dilattion = 1

In [110]:
input_data

tensor([[[[-1.3021, -1.0841],
          [ 1.5359, -2.0545]]]])

In [111]:
weights

tensor([[[[ 0.2128, -0.5075],
          [-1.0375,  0.3759]]]])

In [5]:
def conv2d_transposed(
    matrix, in_channels, out_channels, kernel_size,
    stride=1, padding=0, output_padding=0, dilation=1,
    bias=True, padding_mode='zeros'
):

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode != 'zeros'):
      raise ValueError('only "zeros" padding_mode in ConvTranspose2d')

    #генерация ядра
    if type(kernel_size) == tuple:
      weights = torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
    if type(kernel_size) == int:
      weights = torch.rand(in_channels, out_channels, kernel_size, kernel_size)

    res_tensor = []

    for l in range(out_channels):

      feature_map = torch.zeros((matrix.shape[1]-1)*stride + dilation * (kernel_size-1)+1, (matrix.shape[2]-1)*stride  + dilation * (kernel_size-1)+1 ) #генерация пустой feature-map
      for c in range (in_channels):

        for i in range (0, matrix.shape[1]):  #проход по всем пикселям изображения
          for j in range (0, matrix.shape[2]):

            val = matrix[c][i][j]
            proizv = val*weights[c][l]

            zero_tensor = torch.zeros((weights.shape[2]-1)*dilation+1, (weights.shape[3]-1)*dilation+1)

            for a in range (0, zero_tensor.shape[0], dilation):
              for b in range (0, zero_tensor.shape[1], dilation):
                zero_tensor[a][b] = proizv[a//dilation][b//dilation]

            res = np.add((zero_tensor), feature_map[i*stride:i*stride+(weights.shape[2]-1)*dilation+1, j*stride:j*stride+(weights.shape[3]-1)*dilation+1])
            feature_map[i*stride:i*stride+(weights.shape[2]-1)*dilation+1, j*stride:j*stride+(weights.shape[3]-1)*dilation+1] = res


      res_tensor.append(np.add(feature_map, np.full((feature_map.shape), bias_val[l])))


    for t in range(len(res_tensor)):
      if output_padding > 0:
        pad_func = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
        res_tensor[t] = pad_func(res_tensor[t])

      res_tensor[t] = res_tensor[t][0+padding:res_tensor[t].shape[0]-padding, 0+padding:res_tensor[t].shape[1]-padding]


    return res_tensor, weights, torch.tensor(bias_val)

In [1]:
def compare(tensor, in_channels, out_channels, kernel_size, stride, padding, output_padding, dilation, bias=True, padding_mode='zeros'):
    myres, kernel, bias_val = conv2d_transposed(
        tensor,
        in_channels=in_channels, out_channels=out_channels, 
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
        )
    torchFunction = torch.nn.ConvTranspose2d(
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride, 
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias, 
        padding_mode=padding_mode,
        )
    torchFunction.weight.data = kernel
    torchFunction.bias.data = bias_val

    result = str(np.round([tensor.tolist() for tensor in myres],2))
    torch_res = str(np.round(torchFunction(tensor).data.numpy(),2))
    print(result==torch_res)

In [6]:
tensor1 = torch.rand(8, 5, 6)
tensor2 = torch.rand(3, 28, 28)
tensor3 = torch.rand(5, 6, 6)
tensor4 = torch.rand(1, 1, 1)

In [8]:
compare(tensor1, in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')
compare(tensor2, in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0, output_padding=0, dilation=3, bias=True, padding_mode='zeros')
compare(tensor3, in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5, output_padding=2, dilation=1, bias=True, padding_mode='zeros')
compare(tensor4, in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')

C:\Users\Vexrina\AppData\Local\Temp\ipykernel_7056\1286083728.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return res_tensor, weights, torch.tensor(bias_val)


True
True
True
True


In [9]:
def cooler_transpconv2d(
  input, in_channels, out_channels, kernel_size,
  transp_stride=1, padding=0, dilation=1, bias=True,
  padding_mode='zeros' 
):
  stride = 1
  
  pad = kernel_size - 1
  result_input = []
  for matr in input:
    zero_tensor = np.zeros((((matr.shape[0]-1)*(transp_stride)+1), ((matr.shape[1]-1)*(transp_stride)+1)))
    for a in range (0, zero_tensor.shape[0], transp_stride):
      for b in range (0, zero_tensor.shape[1], transp_stride):
        zero_tensor[a][b] = matr[a//(transp_stride)][b//(transp_stride)]

    pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
    result_input.append(pad_matr)
  input = torch.tensor(result_input)
  if bias == True:
    bias_val = torch.rand(out_channels)
  else:
    bias_val = torch.zeros(out_channels)
    
  if (padding_mode == 'zeros'):
    pad = torch.nn.ZeroPad2d(padding)
    input = pad(input)
  if (padding_mode == 'reflect'):
    pad = torch.nn.ReflectionPad2d(padding)
    input = pad(input)
  if (padding_mode == 'replicate'):
    pad = torch.nn.ReplicationPad2d(padding)
    input = pad(input)
  if (padding_mode == 'circular'):
    pad = torch.nn.CircularPad2d(padding)
    input = pad(input)

  weights = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

  weights_for_transpose = []
  for j in range(out_channels):
    weights_in = []
    for i in range(in_channels):
      weights_in.append(np.flip(np.array(weights[j][i])))
    weights_for_transpose.append(weights_in)

  weights_for_transpose = torch.tensor(weights_for_transpose)
  weights_for_transpose = weights_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)



  res_tensor = []
  for l in range(out_channels):
    feature_map = np.array([])
    for i in range (0, input.shape[1]-((weights.shape[2]-1)*dilation+1)+1, stride):
      for j in range (0, input.shape[2]-((weights.shape[3]-1)*dilation+1)+1, stride):
        summa = 0
        for c in range (in_channels):
          val = input[c][i:i+(weights.shape[2]-1)*dilation+1:dilation, j:j+(weights.shape[3]-1)*dilation+1:dilation]
          mini_sum = (val*weights[l][c]).sum()
          summa = summa + mini_sum
        feature_map = np.append(feature_map, float(summa + bias_val[l]))
    res_tensor.append(feature_map.reshape((input.shape[1]-((weights.shape[2]-1)*dilation+1))//stride+1, (input.shape[2]-((weights.shape[3]-1)*dilation+1))//stride+1))


  return np.array(res_tensor), torch.tensor(np.array(weights_for_transpose)), torch.tensor(np.array(bias_val))

In [15]:
def cooler_compare(tensor, in_channels, out_channels, kernel_size, stride, transp_stride, bias=True,):
    myres, kernel, bias_val = cooler_transpconv2d(
        tensor,
        in_channels=in_channels,
        out_channels=out_channels, 
        kernel_size=kernel_size, 
        transp_stride=transp_stride, 
        bias=bias,
        )
    torchFunction = torch.nn.ConvTranspose2d(
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=kernel_size,
        stride=stride,
        bias=bias,
        )
    torchFunction.weight.data = kernel
    torchFunction.bias.data = bias_val

    result = str(np.round(myres, 2))
    torch_res = str(np.round(torchFunction(tensor).data.numpy(),2))
    print(result==torch_res)

In [10]:
tensor1 = torch.rand(3, 5, 6)
tensor2 = torch.rand(1, 28, 28)
tensor3 = torch.rand(7, 10, 10)

In [18]:
cooler_compare(tensor1, 3,1,3,2,2,True)
cooler_compare(tensor2, 1,2,4,3,3,True)
cooler_compare(tensor3, 7,1,3,5,5,True)

True
True
True
